## Preprocessing danych

Na tym etapie przygotowano dane do trenowania modeli (ML i DL). Wykonano czyszczenie danych, konwersję typów, transformacje wybranych zmiennych oraz imputację braków w sposób ograniczający ryzyko wycieku informacji (data leakage).

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("../data/raw/train.csv")
test_df = pd.read_csv("../data/raw/test.csv")

C:\Users\mikol\AppData\Local\Temp\ipykernel_30956\1428377002.py:1: DtypeWarning: Columns (0: Monthly_Balance) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("../data/raw/train.csv")


In [3]:
drop_cols = ['ID','Customer_ID','Name','SSN']

train_df = train_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)

W pierwszym kroku usunięto kolumny identyfikacyjne oraz zawierające dane wrażliwe, które nie wnoszą informacji predykcyjnej.

In [4]:
num_like_cols = [
    'Age',
    'Annual_Income',
    'Outstanding_Debt',
    'Num_of_Loan',
    'Changed_Credit_Limit',
    'Amount_invested_monthly'
]

for col in num_like_cols:
    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
    test_df[col]  = pd.to_numeric(test_df[col], errors='coerce')

Kolejnym krokiem wykonano konwersję wybranych kolumn zapisanych jako tekst do typu numerycznego. Wartości, których nie dało się poprawnie sparsować, ustawiono jako `NaN` w celu dalszej imputacji.

In [5]:
train_df.loc[(train_df['Age'] < 18) | (train_df['Age'] > 100), 'Age'] = np.nan
test_df.loc[(test_df['Age'] < 18) | (test_df['Age'] > 100), 'Age'] = np.nan

Dodatkowo zastosowano walidację zmiennej `Age` (zakres 18–100), a wartości spoza zakresu potraktowano jako brakujące.

In [6]:
def parse_credit_history(age_str):
    if pd.isna(age_str):
        return np.nan
    try:
        years = int(age_str.split('Years')[0].strip())
        months = int(age_str.split('and')[1].split('Months')[0].strip())
        return years * 12 + months
    except:
        return np.nan

train_df['Credit_History_Months'] = train_df['Credit_History_Age'].apply(parse_credit_history)
test_df['Credit_History_Months']  = test_df['Credit_History_Age'].apply(parse_credit_history)

train_df = train_df.drop(columns=['Credit_History_Age'])
test_df  = test_df.drop(columns=['Credit_History_Age'])

Przekształcono zmienną tekstową `Credit_History_Age` do postaci liczbowej `Credit_History_Months` (łączna liczba miesięcy).

Dodatkowo uwzględniono obsługę brakujących oraz niepoprawnie
sformatowanych wartości poprzez zwracanie `NaN`.
Oryginalna kolumna tekstowa została usunięta po zakończeniu transformacji.

In [7]:
num_cols = train_df.select_dtypes(include='number').columns

for col in num_cols:
    median = train_df[col].median()
    train_df[col] = train_df[col].fillna(median)
    test_df[col]  = test_df[col].fillna(median)

In [8]:
cat_cols = train_df.select_dtypes(include=['object', 'string']).columns

cat_cols = [c for c in cat_cols if c != 'Credit_Score']

for col in cat_cols:
    mode = train_df[col].mode(dropna=True)[0]
    train_df[col] = train_df[col].fillna(mode)

    if col in test_df.columns:
        test_df[col] = test_df[col].fillna(mode)


Braki w zmiennych numerycznych uzupełniono medianą obliczoną na zbiorze treningowym (odporność na wartości odstające).
Dla zmiennych kategorycznych zastosowano najczęściej występującą wartość (modę), z wyłączeniem zmiennej docelowej `Credit_Score`.
Te same wartości imputacyjne wykorzystano następnie dla zbioru testowego, aby ograniczyć ryzyko wycieku informacji.

In [13]:
dirty_numeric_cols = ['Num_of_Delayed_Payment']

for col in dirty_numeric_cols:
    train_df[col] = train_df[col].astype(str).str.replace('_', '', regex=False)
    test_df[col]  = test_df[col].astype(str).str.replace('_', '', regex=False)

    train_df[col] = pd.to_numeric(train_df[col], errors='coerce')
    test_df[col]  = pd.to_numeric(test_df[col], errors='coerce')

In [14]:
med = train_df['Num_of_Delayed_Payment'].median()
train_df['Num_of_Delayed_Payment'] = train_df['Num_of_Delayed_Payment'].fillna(med)
test_df['Num_of_Delayed_Payment']  = test_df['Num_of_Delayed_Payment'].fillna(med)

Kolumna `Num_of_Delayed_Payment` zawierała dane liczbowe zapisane
w niepoprawnym formacie tekstowym (np. wartości w postaci `"8_"`),
co powodowało, że była traktowana jako typ `str`.

Najpierw usunięto znaki specjalne i podjęto próbę konwersji do typu numerycznego.
Niesparsowane wartości ustawiono jako `NaN`, a następnie uzupełniono medianą z treningu.

In [16]:
train_df['Monthly_Balance'] = pd.to_numeric(
    train_df['Monthly_Balance'],
    errors='coerce'
)

test_df['Monthly_Balance'] = pd.to_numeric(
    test_df['Monthly_Balance'],
    errors='coerce'
)

In [17]:
mb_median = train_df['Monthly_Balance'].median()

train_df['Monthly_Balance'] = train_df['Monthly_Balance'].fillna(mb_median)
test_df['Monthly_Balance']  = test_df['Monthly_Balance'].fillna(mb_median)

Zmienna `Monthly_Balance` była zapisana jako `object`, co wskazywało na wartości nienumeryczne.
Wykonano konwersję do typu liczbowego (`errors='coerce'`), a braki uzupełniono medianą z treningu.

In [39]:
train_df.to_csv("../data/processed/train_clean.csv", index=False)
test_df.to_csv("../data/processed/test_clean.csv", index=False)

## Podsumowanie preprocessingu

Na etapie preprocessingu:
- usunięto kolumny identyfikacyjne i wrażliwe,
- skonwertowano wybrane cechy do typów numerycznych oraz wykonano walidację zakresów,
- przekształcono zmienną `Credit_History_Age` do postaci liczbowej,
- uzupełniono braki danych (mediana dla cech numerycznych, moda dla kategorycznych) z zachowaniem zasad braku wycieku informacji.

Oczyszczone dane zapisano do plików w folderze `processed` i zostaną użyte w kolejnych notebookach modelowania.